# 04 - LLM Integration mit LangChain

Integration der Retrieval-Methoden mit einem Large Language Model für vollständige RAG-Pipeline.

## Ziele:
- LangChain für RAG-Pipeline einrichten
- Vector- und Graph-Retrieval mit LLM verbinden
- Antworten generieren und vergleichen
- RAG-Pipeline für Evaluation vorbereiten

In [61]:
# Imports
import json
from http.client import responses

import pandas as pd
import numpy as np
import pickle
import time
from pathlib import Path
import os
from typing import List, Dict

# LangChain Imports (korrigiert)
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

print("Imports erfolgreich")

Imports erfolgreich


## OpenAI API Setup

**Wichtig:** Es wird ein OpenAI API Key benötigt.
- Der Key muss in einer Datei `openai_key.txt` im Root-Verzeichnis des Projekts gespeichert werden

Falls kein OpenAI Key verfügbar ist, nutzen wir ein lokales Modell als Fallback.

In [62]:
# OpenAI API Key Setup
OPENAI_API_KEY = ""

# Versuche API Key aus Datei zu lesen
try:
    key_file_path = "../openai_key.txt"  # Pfad zur Key-Datei im Root
    with open(key_file_path, 'r', encoding='utf-8') as f:
        OPENAI_API_KEY = f.read().strip()
    print("OpenAI API Key aus Datei geladen")
except FileNotFoundError:
    print("openai_key.txt nicht gefunden im Root-Verzeichnis")
    # Fallback: Umgebungsvariable
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
    if OPENAI_API_KEY:
        print("OpenAI API Key aus Umgebungsvariable geladen")
except Exception as e:
    print(f"Fehler beim Laden des API Keys: {e}")

# API Key konfigurieren
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    print("OpenAI API Key konfiguriert")
    USE_OPENAI = True
else:
    print("Kein OpenAI API Key gefunden")
    print("Verwende lokales Modell als Fallback")
    print("Erstelle openai_key.txt im Root-Verzeichnis mit deinem API Key")
    USE_OPENAI = False

OpenAI API Key aus Datei geladen
OpenAI API Key konfiguriert


## Daten und vorherige Ergebnisse laden

In [63]:
# FAQ-Korpus laden
with open('../data/faq_korpus.json', 'r', encoding='utf-8') as f:
    faq_documents = json.load(f)

# Testfragen laden
test_questions = pd.read_csv('../data/fragenliste.csv')

# Vector Retrieval Ergebnisse laden
vector_df = pd.read_csv('../results/vector_retrieval_results.csv')

# Graph Retrieval Ergebnisse laden
graph_df = pd.read_csv('../results/graph_retrieval_results.csv')

print(f"{len(faq_documents)} FAQ-Dokumente geladen")
print(f"{len(test_questions)} Testfragen geladen")
print(f"Vector Retrieval Ergebnisse geladen")
print(f"Graph Retrieval Ergebnisse geladen")

15 FAQ-Dokumente geladen
12 Testfragen geladen
Vector Retrieval Ergebnisse geladen
Graph Retrieval Ergebnisse geladen


## LLM initialisieren

In [64]:
# LLM initialisieren
if USE_OPENAI:
    # OpenAI ChatGPT
    llm = ChatOpenAI(
        model="gpt-4.1-nano",
        temperature=0.1,  # Niedrige Temperatur für konsistente Antworten
        max_tokens=500
    )
    print("OpenAI ChatGPT-4.1-nano initialisiert")
else:
    # Fallback: Dummy LLM für Demo
    class DummyLLM:
        def __call__(self, prompt):
            return "Demo-Antwort: Diese Antwort wurde ohne echtes LLM generiert, da kein OpenAI API Key verfügbar ist."

    llm = DummyLLM()
    print("Dummy-LLM initialisiert (Demo-Modus)")

OpenAI ChatGPT-4.1-nano initialisiert


## RAG-Prompt Templates definieren

In [65]:
# Prompt Template für RAG
rag_prompt_template = """Du bist ein hilfsreicher Assistent für Fragen zu Retrieval-Augmented Generation (RAG) und verwandten Technologien.

Kontext-Informationen:
{context}

Frage: {question}

Anweisungen:
- Beantworte die Frage basierend auf den bereitgestellten Kontext-Informationen
- Wenn die Informationen nicht ausreichen, sage das ehrlich
- Halte deine Antwort präzise und hilfreich
- Verwende die Fachbegriffe aus dem Kontext korrekt

Antwort:"""

# Prompt für Vergleich ohne Kontext (als Baseline)
baseline_prompt_template = """Du bist ein hilfsreicher Assistent für Fragen zu Retrieval-Augmented Generation (RAG) und verwandten Technologien.

Frage: {question}

Beantworte die Frage basierend auf deinem allgemeinen Wissen über RAG, Machine Learning und AI.

Antwort:"""

print("Prompt Templates definiert")

Prompt Templates definiert


## Vector Retrieval RAG-Pipeline

In [66]:
# Vector Retrieval laden
import faiss
from sentence_transformers import SentenceTransformer

# FAISS Index und Metadaten laden
faiss_index = faiss.read_index('../data/faiss_index.bin')
with open('../data/vector_metadata.pkl', 'rb') as f:
    vector_metadata = pickle.load(f)

embedding_model = SentenceTransformer(vector_metadata['model_name'])

def vector_rag_pipeline(question: str, top_k: int = 3) -> Dict:
    """
    Vollständige Vector RAG Pipeline
    """
    start_time = time.time()

    # 1. Query Embedding generieren
    query_embedding = embedding_model.encode([question], convert_to_numpy=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    # 2. Ähnliche Dokumente finden
    scores, indices = faiss_index.search(query_embedding.astype('float32'), top_k)

    # 3. Kontext zusammenstellen
    retrieved_docs = []
    context_parts = []

    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        if idx != -1:
            doc_meta = vector_metadata['document_metadata'][idx]
            retrieved_docs.append(doc_meta)
            context_parts.append(f"Dokument {i+1}: {doc_meta['answer']}")

    context = "\n\n".join(context_parts)

    # 4. LLM-Antwort generieren (KORRIGIERT)
    if USE_OPENAI:
        prompt = rag_prompt_template.format(context=context, question=question)
        # Korrigierter LLM-Aufruf für ChatOpenAI
        from langchain_core.messages import HumanMessage
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)
        answer = response.content
    else:
        answer = f"Demo-Antwort basierend auf {len(retrieved_docs)} Dokumenten: {question}"

    retrieval_time = time.time() - start_time

    return {
        'question': question,
        'answer': answer,
        'retrieved_docs': retrieved_docs,
        'retrieval_scores': scores[0].tolist() if len(scores[0]) > 0 else [],
        'retrieval_time': retrieval_time,
        'method': 'vector'
    }

print("Vector RAG Pipeline implementiert")

Vector RAG Pipeline implementiert


## Graph Retrieval RAG-Pipeline

In [67]:
# Graph Retrieval RAG Pipeline
from neo4j import GraphDatabase
import spacy

# Neo4j wieder verbinden
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    nlp = spacy.load("de_core_news_sm") if spacy.util.is_package("de_core_news_sm") else spacy.load("en_core_web_sm")
    GRAPH_AVAILABLE = True
    print("Neo4j-Verbindung und Spacy geladen")
except Exception as e:
    print(f"Graph Retrieval nicht verfügbar: {e}")
    GRAPH_AVAILABLE = False

def extract_query_terms(text):
    """Extrahiert Suchbegriffe aus der Query"""
    doc = nlp(text)
    terms = []

    # Named Entities
    for ent in doc.ents:
        if len(ent.text.strip()) > 2:
            terms.append(ent.text.strip().lower())

    # Wichtige Konzepte
    for chunk in doc.noun_chunks:
        term = chunk.text.strip().lower()
        if len(term) > 3 and not term.startswith(('der', 'die', 'das')):
            terms.append(term)

    # Fallback: einfache Wörter
    simple_terms = [word.lower().strip() for word in text.split()
                   if len(word) > 3 and word.lower() not in ['sind', 'eine', 'wie', 'was']]

    return list(set(terms + simple_terms))

def graph_rag_pipeline(question: str, top_k: int = 3) -> Dict:
    """
    Vollständige Graph RAG Pipeline
    """
    if not GRAPH_AVAILABLE:
        return {
            'question': question,
            'answer': "Graph Retrieval nicht verfügbar - Neo4j nicht verbunden",
            'retrieved_docs': [],
            'retrieval_scores': [],
            'retrieval_time': 0,
            'method': 'graph'
        }

    start_time = time.time()

    # 1. Query-Begriffe extrahieren
    query_terms = extract_query_terms(question)

    if not query_terms:
        query_terms = [question.lower()]

    # 2. Graph-Suche durchführen
    with driver.session() as session:
        cypher_query = """
        MATCH (d:Document)
        OPTIONAL MATCH (d)-[:MENTIONS]->(c:Concept)
        OPTIONAL MATCH (d)-[:CONTAINS]->(e:Entity)

        WITH d,
             collect(DISTINCT c.name) as concepts,
             collect(DISTINCT e.name) as entities

        WITH d, concepts, entities,
             REDUCE(score = 0, term IN $query_terms |
                 score +
                 CASE WHEN term IN concepts THEN 3 ELSE 0 END +
                 CASE WHEN term IN entities THEN 2 ELSE 0 END +
                 CASE WHEN toLower(d.question) CONTAINS term THEN 2 ELSE 0 END +
                 CASE WHEN toLower(d.answer) CONTAINS term THEN 1 ELSE 0 END
             ) as relevance_score

        WHERE relevance_score > 0

        RETURN d.id as doc_id,
               d.question as question,
               d.answer as answer,
               d.category as category,
               relevance_score

        ORDER BY relevance_score DESC
        LIMIT $top_k
        """

        result = session.run(cypher_query, {
            'query_terms': query_terms,
            'top_k': top_k
        })

        # 3. Kontext zusammenstellen
        retrieved_docs = []
        context_parts = []
        scores = []

        for i, record in enumerate(result):
            doc_info = {
                'id': record['doc_id'],
                'question': record['question'],
                'answer': record['answer'],
                'category': record['category']
            }
            retrieved_docs.append(doc_info)
            context_parts.append(f"Dokument {i+1}: {doc_info['answer']}")
            scores.append(record['relevance_score'])

        context = "\n\n".join(context_parts)

    # 4. LLM-Antwort generieren (KORRIGIERT)
    if USE_OPENAI and context:
        prompt = rag_prompt_template.format(context=context, question=question)
        # Korrigierter LLM-Aufruf für ChatOpenAI
        from langchain_core.messages import HumanMessage
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)
        answer = response.content
    elif context:
        answer = f"Demo-Antwort basierend auf {len(retrieved_docs)} Graph-Dokumenten: {question}"
    else:
        answer = "Keine relevanten Dokumente im Knowledge Graph gefunden."

    retrieval_time = time.time() - start_time

    return {
        'question': question,
        'answer': answer,
        'retrieved_docs': retrieved_docs,
        'retrieval_scores': scores,
        'retrieval_time': retrieval_time,
        'method': 'graph'
    }

print("Graph RAG Pipeline implementiert")

Neo4j-Verbindung und Spacy geladen
Graph RAG Pipeline implementiert


## Baseline ohne Retrieval

In [68]:
def baseline_pipeline(question: str) -> Dict:
    """
    Baseline: LLM ohne Retrieval (nur allgemeines Wissen)
    """
    start_time = time.time()

    if USE_OPENAI:
        prompt = baseline_prompt_template.format(question=question)
        # Korrigierter LLM-Aufruf für ChatOpenAI
        from langchain_core.messages import HumanMessage
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)
        answer = response.content
    else:
        answer = f"Demo-Baseline-Antwort ohne Retrieval: {question}"

    generation_time = time.time() - start_time

    return {
        'question': question,
        'answer': answer,
        'retrieved_docs': [],
        'retrieval_scores': [],
        'retrieval_time': generation_time,
        'method': 'baseline'
    }

print("Baseline Pipeline implementiert")

Baseline Pipeline implementiert


## Test der RAG-Pipelines

In [69]:
# Test mit einer Beispielfrage
test_question = "Was ist RAG?"

print("=== RAG-PIPELINE TESTS ===")
print(f"Testfrage: '{test_question}'")

# Vector RAG
print("\n--- VECTOR RAG ---")
vector_result = vector_rag_pipeline(test_question)
print(f"Retrieval Zeit: {vector_result['retrieval_time']:.3f}s")
print(f"Gefundene Dokumente: {len(vector_result['retrieved_docs'])}")
print(f"Antwort: {vector_result['answer'][:200]}...")

# Graph RAG
print("\n--- GRAPH RAG ---")
graph_result = graph_rag_pipeline(test_question)
print(f"Retrieval Zeit: {graph_result['retrieval_time']:.3f}s")
print(f"Gefundene Dokumente: {len(graph_result['retrieved_docs'])}")
print(f"Antwort: {graph_result['answer'][:200]}...")

# Baseline
print("\n--- BASELINE ---")
baseline_result = baseline_pipeline(test_question)
print(f"Generation Zeit: {baseline_result['retrieval_time']:.3f}s")
print(f"Antwort: {baseline_result['answer'][:200]}...")

=== RAG-PIPELINE TESTS ===
Testfrage: 'Was ist RAG?'

--- VECTOR RAG ---
Retrieval Zeit: 1.059s
Gefundene Dokumente: 3
Antwort: RAG (Retrieval-Augmented Generation) ist eine Technik, die Large Language Models mit externen Wissensquellen verbindet. Das Modell kann relevante Informationen aus einer Wissensbasis abrufen und diese...

--- GRAPH RAG ---
Retrieval Zeit: 0.829s
Gefundene Dokumente: 3
Antwort: RAG (Retrieval-Augmented Generation) ist eine Technik, die Large Language Models mit externen Wissensquellen verbindet. Das Modell kann relevante Informationen aus einer Wissensbasis abrufen und diese...

--- BASELINE ---
Generation Zeit: 1.400s
Antwort: RAG (Retrieval-Augmented Generation) ist eine fortschrittliche Methode im Bereich der Künstlichen Intelligenz, die die Stärken von Retrieval-Systemen und generativen Sprachmodellen kombiniert. Dabei w...


## Vollständige Evaluation aller Testfragen

In [70]:
# Alle Testfragen durch alle Pipelines laufen lassen
print("=== VOLLSTÄNDIGE RAG-PIPELINE EVALUATION ===")

all_results = []

for idx, row in test_questions.iterrows():
    question = row['question']
    question_id = row['id']
    difficulty = row['difficulty']

    print(f"🔄 Verarbeite {question_id} ({difficulty}): {question[:50]}...")

    # Vector RAG
    vector_result = vector_rag_pipeline(question)
    all_results.append({
        'question_id': question_id,
        'question': question,
        'difficulty': difficulty,
        'method': 'vector',
        'answer': vector_result['answer'],
        'num_retrieved': len(vector_result['retrieved_docs']),
        'retrieval_time': vector_result['retrieval_time'],
        'avg_retrieval_score': np.mean(vector_result['retrieval_scores']) if vector_result['retrieval_scores'] else 0
    })

    # Graph RAG
    graph_result = graph_rag_pipeline(question)
    all_results.append({
        'question_id': question_id,
        'question': question,
        'difficulty': difficulty,
        'method': 'graph',
        'answer': graph_result['answer'],
        'num_retrieved': len(graph_result['retrieved_docs']),
        'retrieval_time': graph_result['retrieval_time'],
        'avg_retrieval_score': np.mean(graph_result['retrieval_scores']) if graph_result['retrieval_scores'] else 0
    })

    # Baseline
    baseline_result = baseline_pipeline(question)
    all_results.append({
        'question_id': question_id,
        'question': question,
        'difficulty': difficulty,
        'method': 'baseline',
        'answer': baseline_result['answer'],
        'num_retrieved': 0,
        'retrieval_time': baseline_result['retrieval_time'],
        'avg_retrieval_score': 0
    })

print(f"{len(all_results)} Antworten generiert ({len(all_results)//3} Fragen × 3 Methoden)")

=== VOLLSTÄNDIGE RAG-PIPELINE EVALUATION ===
Verarbeite q001 (easy): Erkläre mir RAG in einfachen Worten...
Verarbeite q002 (medium): Was sind die Unterschiede zwischen Vektor- und Gra...
Verarbeite q003 (medium): Welche Evaluationsmetriken sollte ich für mein RAG...
Verarbeite q004 (hard): Wie optimiere ich die Performance meines RAG-Syste...
Verarbeite q005 (easy): Was kostet der Betrieb eines RAG-Systems?...
Verarbeite q006 (hard): Wie implementiere ich Hybrid Retrieval?...
Verarbeite q007 (medium): Welche Vector Database sollte ich wählen?...
Verarbeite q008 (medium): Was sind häufige Probleme bei RAG und wie löse ich...
Verarbeite q009 (hard): Wie funktioniert RAG mit Knowledge Graphs?...
Verarbeite q010 (medium): Was ist der optimale Chunk-Size für meine Dokument...
Verarbeite q011 (easy): Welche Zukunftstrends gibt es bei RAG?...
Verarbeite q012 (hard): Wie verbessere ich meine Embedding-Qualität?...
36 Antworten generiert (12 Fragen × 3 Methoden)


## Ergebnisse analysieren

In [71]:
# Ergebnisse in DataFrame konvertieren
results_df = pd.DataFrame(all_results)

print("=== RAG-PIPELINE ANALYSE ===")
print(f"Gesamte Antworten: {len(results_df)}")

# Analyse nach Methode
method_stats = results_df.groupby('method').agg({
    'retrieval_time': ['mean', 'std'],
    'num_retrieved': 'mean',
    'avg_retrieval_score': 'mean'
}).round(3)

print("\n=== PERFORMANCE NACH METHODE ===")
print(method_stats)

# Analyse nach Schwierigkeit
difficulty_stats = results_df.groupby(['difficulty', 'method'])['retrieval_time'].mean().unstack().round(3)
print("\n=== RETRIEVAL-ZEIT NACH SCHWIERIGKEIT UND METHODE ===")
print(difficulty_stats)

# Erfolgsrate (Dokumente gefunden)
success_rate = results_df[results_df['method'] != 'baseline'].groupby('method')['num_retrieved'].apply(lambda x: (x > 0).mean()).round(3)
print("\n=== ERFOLGSRATE (Dokumente gefunden) ===")
print(success_rate)

=== RAG-PIPELINE ANALYSE ===
Gesamte Antworten: 36

=== PERFORMANCE NACH METHODE ===
         retrieval_time        num_retrieved avg_retrieval_score
                   mean    std          mean                mean
method                                                          
baseline          3.007  1.080         0.000               0.000
graph             0.764  0.597         1.917               3.417
vector            1.117  0.333         3.000               0.520

=== RETRIEVAL-ZEIT NACH SCHWIERIGKEIT UND METHODE ===
method      baseline  graph  vector
difficulty                         
easy           2.473  0.853   1.030
hard           3.247  0.607   1.201
medium         3.137  0.837   1.101

=== ERFOLGSRATE (Dokumente gefunden) ===
method
graph     0.667
vector    1.000
Name: num_retrieved, dtype: float64


## Beispiel-Antworten vergleichen

In [72]:
# Beispiel-Vergleich für eine mittelschwere Frage
sample_question_id = 'q002'  # "Was sind die Unterschiede zwischen Vektor- und Graph-Retrieval?"

sample_results = results_df[results_df['question_id'] == sample_question_id]

print("=== ANTWORTEN-VERGLEICH ===")
print(f"Frage: {sample_results.iloc[0]['question']}")

for _, result in sample_results.iterrows():
    print(f"\n--- {result['method'].upper()} ---")
    print(f"Zeit: {result['retrieval_time']:.3f}s")
    print(f"Gefundene Docs: {result['num_retrieved']}")
    print(f"Antwort: {result['answer'][:300]}...")

=== ANTWORTEN-VERGLEICH ===
Frage: Was sind die Unterschiede zwischen Vektor- und Graph-Retrieval?

--- VECTOR ---
Zeit: 1.265s
Gefundene Docs: 3
Antwort: Vektor-Retrieval basiert auf der Umwandlung von Dokumenten und Anfragen in hochdimensionale Embeddings, wobei die Ähnlichkeit durch Metriken wie Cosinus-Ähnlichkeit gemessen wird, um semantische Beziehungen zu erfassen. Graph-Retrieval hingegen nutzt Graphstrukturen, um Beziehungen zwischen Entitäte...

--- GRAPH ---
Zeit: 1.562s
Gefundene Docs: 3
Antwort: Vektor-Retrieval nutzt neuronale Embeddings, um semantische Ähnlichkeiten zwischen Anfragen und Dokumenten zu erkennen, und basiert auf Dense Retrieval, das für semantische Suche geeignet ist. Graph-Retrieval hingegen erfasst Beziehungen zwischen Entitäten durch Graphstrukturen, ermöglicht komplexes...

--- BASELINE ---
Zeit: 2.200s
Gefundene Docs: 0
Antwort: Der Unterschied zwischen Vektor- und Graph-Retrieval liegt in der Art und Weise, wie Informationen gesucht und organisiert w

## Ergebnisse speichern

In [73]:
# RAG-Pipeline Ergebnisse speichern
results_df.to_csv('../results/rag_pipeline_results.csv', index=False, encoding='utf-8')

# Detaillierte Antworten für spätere Analyse speichern
detailed_results = {}
for idx, row in results_df.iterrows():
    key = f"{row['question_id']}_{row['method']}"
    detailed_results[key] = {
        'question': row['question'],
        'answer': row['answer'],
        'method': row['method'],
        'difficulty': row['difficulty'],
        'retrieval_time': row['retrieval_time']
    }

with open('../results/detailed_answers.json', 'w', encoding='utf-8') as f:
    json.dump(detailed_results, f, ensure_ascii=False, indent=2)

print("RAG-Pipeline Ergebnisse gespeichert:")
print("  - ../results/rag_pipeline_results.csv")
print("  - ../results/detailed_answers.json")

RAG-Pipeline Ergebnisse gespeichert:
  - ../results/rag_pipeline_results.csv
  - ../results/detailed_answers.json


In [74]:
print("\n=== LLM INTEGRATION ABGESCHLOSSEN ===")
print("RAG-Pipelines für Vector, Graph und Baseline implementiert")
print(f"{len(results_df)} Antworten generiert")
print("Detaillierte Ergebnisse gespeichert")
print(f"Durchschnittliche Antwortzeit: {results_df['retrieval_time'].mean():.3f}s")

# Aufräumen
if GRAPH_AVAILABLE:
    driver.close()
    print("Neo4j-Verbindung geschlossen")


=== LLM INTEGRATION ABGESCHLOSSEN ===
RAG-Pipelines für Vector, Graph und Baseline implementiert
36 Antworten generiert
Detaillierte Ergebnisse gespeichert
Durchschnittliche Antwortzeit: 1.629s

Neo4j-Verbindung geschlossen
